In [3]:
import sys
sys.path.insert(0, '/burg/hblab/users/hg2604/Projects/Selex_X_Genome/scripts')
from parse_motifcentral_json import MOTIFCENTRAL 

In [6]:
# MOTIFCENTRAL is a list of JSON objects. Each JSON object corresponds to a single motif.
MOTIFCENTRAL[0]


dict_keys(['modelSettings', 'coefficients', 'metadata'])

In [9]:
MOTIFCENTRAL[0]['modelSettings'] 

{'letterOrder': 'ACGT',
 'bindingModes': [{'fitLogActivity': True,
   'flankLength': 0,
   'singleStrand': False,
   'size': 12,
   'dinucleotideDistance': 0,
   'componentName': 'Binding mode 1',
   'positionBias': False,
   'modifications': []}],
 'letterComplement': 'C-G,A-T'}

In [ ]:
MOTIFCENTRAL[0]['coefficients']['bindingModes']  # List of binding modes fit to an experiment

In [16]:
len(MOTIFCENTRAL[0]['coefficients']['bindingModes'])

1

In [18]:
MOTIFCENTRAL[0]['coefficients']['bindingModes'][0].keys()

dict_keys(['mononucleotide', 'activity', 'dinucleotide', 'positionBias', 'modifications'])

In [26]:
# This has the deltadeltaGs (betas) 'ACGT', to get affinity you need to sum and exponentiate
MOTIFCENTRAL[0]['coefficients']['bindingModes'][0]['mononucleotide'] 

[-0.021149277379063136,
 -0.29650566484723484,
 0,
 -0.15711447637966103,
 -0.15059044429471768,
 -0.7097440179672816,
 0,
 -0.40857968097581576,
 -0.39110897459993765,
 -2.0889344326278803,
 0,
 -2.655015858709927,
 -2.9884769720629727,
 -2.4628947898253664,
 -2.237664556208653,
 0,
 -5.315908182560314,
 0,
 -5.476742305090323,
 -5.947622539961063,
 0,
 -3.8666344168939766,
 -4.490071841612913,
 -3.6795361372751927,
 -5.05043783692896,
 0,
 -3.053265185660482,
 -1.933908571871246,
 -2.032721047414352,
 -3.2650605843879994,
 0,
 -5.123451166831984,
 -1.212232750108413,
 -1.6334842868484785,
 -2.2190249761198473,
 0,
 -5.484222829475819,
 -4.790194070201455,
 0,
 -3.6188465758338424,
 0,
 -0.7199378673195866,
 -0.2593847753681384,
 -0.7176682693863778,
 -0.9702250248223461,
 -0.2941006299827045,
 -0.7546996767952994,
 0]

In [25]:
MOTIFCENTRAL[0]['coefficients']['bindingModes'][0]['modifications']

[]

In [32]:
import numpy as np 
numpy_motif = np.array(MOTIFCENTRAL[0]['coefficients']['bindingModes'][0]['mononucleotide']).reshape(4, 12,order='F')


array([[-0.02114928, -0.15059044, -0.39110897, -2.98847697, -5.31590818,
         0.        , -5.05043784, -2.03272105, -1.21223275, -5.48422283,
         0.        , -0.97022502],
       [-0.29650566, -0.70974402, -2.08893443, -2.46289479,  0.        ,
        -3.86663442,  0.        , -3.26506058, -1.63348429, -4.79019407,
        -0.71993787, -0.29410063],
       [ 0.        ,  0.        ,  0.        , -2.23766456, -5.47674231,
        -4.49007184, -3.05326519,  0.        , -2.21902498,  0.        ,
        -0.25938478, -0.75469968],
       [-0.15711448, -0.40857968, -2.65501586,  0.        , -5.94762254,
        -3.67953614, -1.93390857, -5.12345117,  0.        , -3.61884658,
        -0.71766827,  0.        ]])

In [33]:
import numpy as np

def seq_to_numpy(seq):
    """Encodes a DNA sequence into a numpy array of shape len(seq) x 4

    Args:
        seq (String): The DNA Sequence "ACGT"
    Returns:
        numpy array of shape len(seq) x 4, encoding the DNA sequence
    """
    numpy_seq = np.zeros((len(seq), 4))
    
    for i in range(0, len(seq)):
        match seq[i]:
            case 'A':
                numpy_seq[i, 0] = 1
            case 'C':
                numpy_seq[i, 1] = 1
            case 'G':
                numpy_seq[i, 2] = 1
            case 'T':
                numpy_seq[i, 3] = 1
    
    return numpy_seq

def score_window(numpy_motif, seq):
    """Scores a window the size of motif 

    Args:
        numpy_motif (numpy array of shape len(seq) x 4): The motif
        seq (String): The DNA Sequence "ACGT" szie of the motif
    Returns:
        score (float): The score of the window
    """
    score = 0
    for i, base in enumerate(seq):
        match base:
            case 'A':
                score += numpy_motif[0, i]
            case 'C':
                score += numpy_motif[1, i]
            case 'G':
                score += numpy_motif[2, i]
            case 'T':
                score += numpy_motif[3, i]
    
    return score

def score_seq_against_motif(seq, motif):
    """
    Given a sequence and a motif, scores the sequence against the motif.

    arguments:
        1. seq (str): sequence to score (ACGT)
        2. motif (list): A list of deldelGs. Lenght = Size of motif * 4. Order = ACGT
    returns:
        1. score (float): the score of the sequence against the motif in log space.
    """
    

    motif_size = int(len(motif)/4)
    numpy_motif = np.array(motif).reshape(4, motif_size, order='F')

    score = 0
    for i in range(0, len(seq)-2):
        score += score_window(numpy_motif, seq[i:i+motif_size])
    
    return score


    
    

In [35]:
score = score_seq_against_motif("ATGCTTTCGACG", MOTIFCENTRAL[0]['coefficients']['bindingModes'][0]['mononucleotide'])

In [36]:
score

array([[ -5.50537211,  -5.02119802,  -2.57356675, -10.51693646],
       [ -5.08669974,  -6.44789324,  -4.01651935,  -4.57637843],
       [  0.        ,  -2.49704933,  -2.97372465, -13.02007933],
       [ -3.77596105,  -5.84111944,  -2.65501586, -11.96964693]])